In [ ]:
# Script to visualize a reaction family tree

In [1]:
import os
import sys
import copy
import json
import pickle

import numpy as np
import rmgpy.chemkin
import rmgpy.tools.uncertainty

import matplotlib.pyplot as plt
import pydot
import matplotlib.image as mpimg
%matplotlib inline


In [2]:
# Load the RMG mechanism

# Load the base model and the covariance matrix
basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)

covariance_file = '/home/moon/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)


In [3]:
reaction_list[794].kinetics

Arrhenius(A=(1.5e+11,'cm^3/(mol*s)'), n=0, Ea=(0,'kcal/mol'), T0=(1,'K'), comment="""Estimated from node Root_Ext-2R!H-R_2R!H->C_4R->C
Multiplied by reaction path degeneracy 3.0""")

In [4]:
# load the model

uncertainty = rmgpy.tools.uncertainty.Uncertainty(output_directory='rmg_uncertainty')
uncertainty.load_model(base_chemkin, dictionary)




# TODO - force the user to provide the input file used to generate the mechanism to ensure databases are really the same
# load the database
# --------------- CAUTION!!! Databases here must match the ones used to generate the mechanism
# note - this cell stalls out on Discovery
thermo_libs = [
    'BurkeH2O2',
    'primaryThermoLibrary',
    'FFCM1(-)',
    'CurranPentane',
    'Klippenstein_Glarborg2016',
    'thermo_DFT_CCSDTF12_BAC',
    'DFT_QCI_thermo',
    'CBS_QB3_1dHR',
]

kinetic_libs = [
    'FFCM1(-)',
    'CurranPentane',
    'combustion_core/version5',
    'Klippenstein_Glarborg2016',
    'BurkeH2O2inArHe',
    'BurkeH2O2inN2',
]
uncertainty.load_database(
    thermo_libraries=thermo_libs,
    kinetics_families='default',
    reaction_libraries=kinetic_libs,
    kinetics_depositories=['training'],
)


In [5]:
# Get the different kinetic and thermo sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

KeyError: 'Root_N-4R->H_4CNOS-u1_N-1R!H->O_4CNOS->O_Ext-4O-R_5R!H-u0'

In [6]:
# Create a giant dictionary with all of the reaction family information in it
auto_gen_families = {}
for family_name in uncertainty.database.kinetics.families.keys():
    if family_name == 'Intra_R_Add_Endocyclic' or family_name == 'Intra_R_Add_Exocyclic':
        continue
    if uncertainty.database.kinetics.families[family_name].auto_generated and family_name not in auto_gen_families.keys():
        auto_gen_families[family_name] = uncertainty.database.kinetics.families[family_name].rules.get_entries()
        auto_gen_families[f'{family_name}_labels'] = [entry.label for entry in uncertainty.database.kinetics.families[family_name].rules.get_entries()]
        auto_gen_families[f'{family_name}_rxn_map'] = uncertainty.database.kinetics.families[family_name].get_reaction_matches(
            thermo_database=uncertainty.database.thermo,
            remove_degeneracy=True,
            get_reverse=True,
            exact_matches_only=False,
            fix_labels=True)

In [7]:
family_name = 'Disproportionation'
for rule in uncertainty.database.kinetics.families[family_name].rules.entries:
    print(rule)

Root
Root_1R!H->C
Root_N-1R!H->C
Root_1R!H->C_Ext-4R-R
Root_1R!H->C_2R!H->O
Root_1R!H->C_N-2R!H->O
Root_N-1R!H->C_4R->H
Root_N-1R!H->C_N-4R->H
Root_1R!H->C_Ext-4R-R_4R->O
Root_1R!H->C_Ext-4R-R_N-4R->O
Root_1R!H->C_2R!H->O_Ext-1C-R
Root_1R!H->C_N-2R!H->O_Ext-2CNS-R
Root_1R!H->C_N-2R!H->O_Ext-1C-R
Root_1R!H->C_N-2R!H->O_4R-u1
Root_1R!H->C_N-2R!H->O_N-4R-u1
Root_N-1R!H->C_4R->H_2R!H->O
Root_N-1R!H->C_4R->H_N-2R!H->O
Root_N-1R!H->C_N-4R->H_4CNOS->O
Root_N-1R!H->C_N-4R->H_N-4CNOS->O
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0
Root_1R!H->C_Ext-4R-R_4R->O_N-5R!H-u0
Root_1R!H->C_Ext-4R-R_N-4R->O_5R!H->S
Root_1R!H->C_Ext-4R-R_N-4R->O_N-5R!H->S
Root_1R!H->C_N-2R!H->O_Ext-2CNS-R_Ext-5R!H-R
Root_1R!H->C_N-2R!H->O_Ext-2CNS-R_4R->H
Root_1R!H->C_N-2R!H->O_Ext-2CNS-R_N-4R->H
Root_1R!H->C_N-2R!H->O_Ext-1C-R_Sp-5R!H-1C
Root_1R!H->C_N-2R!H->O_Ext-1C-R_N-Sp-5R!H-1C
Root_1R!H->C_N-2R!H->O_4R-u1_4R->C
Root_1R!H->C_N-2R!H->O_4R-u1_N-4R->C
Root_1R!H->C_N-2R!H->O_N-4R-u1_Sp-2CNS-1C
Root_1R!H->C_N-2R!H->O_N-4R-u1_N-Sp-

In [8]:
for group in uncertainty.database.kinetics.families[family_name].groups.entries:
    print(group)

Root
Root_1R!H->C
Root_1R!H->C_Ext-4R-R
Root_1R!H->C_Ext-4R-R_4R->O
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_Ext-6R!H-R
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_2R!H->C
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_2R!H->C_Ext-5R!H-R
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_2R!H->C_Ext-5R!H-R_Ext-7R!H-R
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_2R!H->C_Ext-5R!H-R_Ext-7R!H-R_Ext-7R!H-R
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_N-2R!H->C
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_N-2R!H->C_5R!H->C
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_N-2R!H->C_N-5R!H->C
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_N-2R!H->C_N-5R!H->C_6R!H-u0
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_Ext-1C-R_N-2R!H->C_N-5R!H->C_N-6R!H-u0
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_2R!H->C
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_2R!H->C_Ext-2C-R
Root_1R!H->C_Ext-4R-R_4R->O_5R!H-u0_2R!H->C_Ext-2C-R_6R!H->C
Root_1R!H->C_Ext-4R-R_4R->O_

# Make the Graph

In [9]:
def get_node_nickname(node):
    parent = node.parent
#     return node.label.split('->')[-1]  # this doesn't work for some reason
    if parent:
        # subtract the length of the parent
        parent_name = node.parent.label
        
        if parent_name != node.label[:len(parent_name)]:
            print(parent_name, node.label)
        return node.label[len(parent_name):]
    return node.label

In [ ]:
# uncertainty.database.kinetics.families[family_name].groups.entries['Root_Ext-1R!H-R'].parent
# uncertainty.database.kinetics.families[family_name].groups.entries['Root'].parent

In [ ]:
# get_node_nickname(uncertainty.database.kinetics.families[family_name].groups.entries['Root'])
# get_node_nickname(uncertainty.database.kinetics.families[family_name].groups.entries['Root_Ext-1R!H-R'])
# get_node_nickname(uncertainty.database.kinetics.families[family_name].groups.entries['Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S'])

In [10]:
def link_children(parent_node):
    if len(parent_node.children) == 0:
        return
    
    for child in parent_node.children:
        link_children(child)
#         graph.add_edge(pydot.Edge(get_node_nickname(parent_node), get_node_nickname(child), color='black'))  #it's a mess if you do it this way
        graph.add_edge(pydot.Edge(str(parent_node.index), str(child.index), color='black'))



#         graph.add_edge(pydot.Edge(get_node_nickname(parent_node), get_node_nickname(child), color='black'))

# def link_children(parent_node):
#     if len(uncertainty.database.kinetics.families[family_name].groups.entries[parent_node].children) == 0:
#         return
    
#     for child in uncertainty.database.kinetics.families[family_name].groups.entries[parent_node].children:
#         link_children(str(child))
#         graph.add_edge(pydot.Edge(str(parent_node.index), str(child.index), color='black'))

In [ ]:
graph = pydot.Dot('Disproportionation_tree', graph_type='digraph', overlap="false")
# graph.set_rankdir('LR')
# graph.set_fontname('sans')
# graph.set_fontsize('10')

# Add nodes
for group in uncertainty.database.kinetics.families[family_name].groups.entries:

    # it's a mess if you do it with nicknames, so use index
#     node_name = get_node_nickname(uncertainty.database.kinetics.families[family_name].groups.entries[group])
#     print(node_name)
#     graph.add_node(pydot.Node(name=node_name))

    if uncertainty.database.kinetics.families[family_name].groups.entries[group].index < 0:
        continue

    graph.add_node(pydot.Node(name=str(uncertainty.database.kinetics.families[family_name].groups.entries[group].index)))
#     print(str(uncertainty.database.kinetics.families[family_name].groups.entries[group].index))

link_children(uncertainty.database.kinetics.families[family_name].groups.entries['Root'])
# link_children('Root')

graph = pydot.graph_from_dot_data(graph.create_dot(prog='dot').decode('utf-8'))[0]

In [ ]:
# change the names of the nodes
for node in graph.get_nodes():
    name = node.get_name()
    for group in uncertainty.database.kinetics.families[family_name].groups.entries:
        if str(uncertainty.database.kinetics.families[family_name].groups.entries[group].index) == name:
            rmgdb_node = uncertainty.database.kinetics.families[family_name].groups.entries[group]
            node.set_label(get_node_nickname(rmgdb_node))
#             print(get_node_nickname())
            break


## Plot the graph

In [ ]:
graph.write_dot('Disproportionation_tree_refit.dot')
graph.write_png('Disproportionation_tree_refit.png')

In [ ]:
img = mpimg.imread('Disproportionation_tree_refit.png')
imgplot = plt.imshow(img)

In [ ]:
# # some ideas for visualization:
# - size the nodes based on the number of training reactions used
# - size the nodes by uncertainty
# - size the nodes by the number of mechanism reactions that use that node
        

In [ ]:
# Richard said there are inbuilt functions for visualizing the groups in RMG. I'd use this if I understood how to read it
# okay, I get it. It's describing the reactants. The group labeled one is a carbon and the groups labeled 2 and 3 are also C
# then the group labeled 3 can be bonded to anything
# okay, but this isn't going to look good on a tree
# Maybe as a tooltip hover option?

uncertainty.database.kinetics.families[family_name].groups.entries['Root_Ext-2R!H-R_2R!H->C_4R->C'].item

In [ ]:
uncertainty.database.kinetics.families[family_name].groups.entries['Root_Ext-2R!H-R_2R!H->C_4R->C'].item